In [1]:
import pandas as pd
from ast import literal_eval

### Reading the result file that we saved from captum training

In [ ]:
df = pd.read_csv('results_test_subset_fine-tuned-xlm_roberta-german-WithNeutral.csv', converters={'Words':literal_eval})
pd.options.display.max_colwidth = 1000
df

Converting the spans(negative object spans and positive object spans columns) which are strings to lists for the code to work properly

In [4]:
df.rename(columns={'negative object_spans':'negative_object_spans','negative spans':'negative_spans','positive object_spans':'positive_object_spans','positive spans':'positive_spans'}, inplace=True)

new_list_positive_object_spans = []
new_list_positive_spans = []
new_list_negative_object_spans = []
new_list_negative_spans =[]

list_positive_object_spans = df.positive_object_spans
list_positive_spans = list(df.positive_spans)
list_negative_object_spans = df.negative_object_spans
list_negative_spans = list(df.negative_spans)

#print(list_negative_object_spans)
#print(list_negative_spans)
#print(list_positive_object_spans)

for item in list_positive_object_spans:
    if not '[' in item:
        new_list_positive_object_spans.append(item.split())
    else:
        new_list_positive_object_spans.append(item)

for item in list_positive_spans:
    if not '[' in item:
        new_list_positive_spans.append(item.split())
    else:
        new_list_positive_spans.append(item)

for item in list_negative_object_spans:
    if not '[' in item:
        new_list_negative_object_spans.append(item.split())
    else:
        new_list_negative_object_spans.append(item)

for item in list_negative_spans:
    if not '[' in item:
        new_list_negative_spans.append(item.split())
    else:
        new_list_negative_spans.append(item)

In [5]:
df['negative_object_spans'] = new_list_negative_object_spans
df['negative_spans'] = new_list_negative_spans
df['positive_object_spans'] = new_list_positive_object_spans
df['positive_spans'] = new_list_positive_spans

Checking every word if present in the spans from original data and if attributions are positive or negative and creating tokens and predicted labels respectively

In [15]:
words_in_spans = []
predicted_labels = []
actual_label = []
for i in range(len(df)):
    for row in df['Words'][i]:
        row['word'] = ''.join(e for e in row['word'] if e.isalnum())
        if row['word'] != '':
            if row['word'] in df['negative_object_spans'][i] or row['word'] in  df['negative_spans'][i] or row['word'] in df['positive_spans'][i] or row['word'] in df['positive_object_spans'][i]:
                words_in_spans.append(row['word'])
                if len(row['attributions']) > 1:
                    max_attribution = max(row['attributions'])
                    if max_attribution > 0:
                        predicted_labels.append(df['Predicted Label'][i])
                        actual_label.append(df['Actual Label'][i])
                    else:
                        predicted_labels.append('O')
                        actual_label.append('O')
                else:
                    if row['attributions'][0] > 0:
                        predicted_labels.append(df['Predicted Label'][i])
                        actual_label.append(df['Actual Label'][i])
                    else:
                        predicted_labels.append('O')
                        actual_label.append('O')
            else:
                words_in_spans.append(row['word'])
                predicted_labels.append('O')
                actual_label.append('O')
    words_in_spans.append(' ')
    actual_label.append(' ')
    predicted_labels.append(' ')
print(len(words_in_spans))
print(len(predicted_labels))
print(len(actual_label))


8378
8378
8378


In [8]:
df_ner_captum = pd.DataFrame(columns=['tokens','actual','predicted'])

In [9]:
df_ner_captum['tokens'] = words_in_spans
df_ner_captum['actual'] = actual_label
df_ner_captum['predicted'] = predicted_labels

In [10]:
def transform_labels(example, column):
    print(str(column))
    for i in range(len(example)):
        if example[column][i] == 0:
            #print(i)
            df_ner_captum.loc[i,column] = 'negative'
        elif example[column][i] == 1:
            df_ner_captum.loc[i,column] = 'O'
        elif example[column][i] == 2:
            df_ner_captum.loc[i,column] = 'positive'

In [11]:
transform_labels(df_ner_captum, 'actual')
transform_labels(df_ner_captum, 'predicted')

actual
predicted


Saving the above results in a file without header and tab seperated. Now with this file we can do NER evaluation using conlleval

In [14]:
df_ner_captum.to_csv('ResultsFromCaptumNEREvaluationXLMRobertaWithNeutralLatestCorrected4th.csv', index =False, sep='\t', header=False)

In [ ]:
### Below are the results of NER Evaluation
#### python conlleval_perl.py -r < ResultsFromCaptumNEREvaluationXLMRobertaWithNeutralLatestCorrected4th.csv   -- command used for NER evaluation
processed 8078 tokens with 1657 phrases; found: 1631 phrases; correct: 1447.
accuracy:  97.40%; precision:  88.72%; recall:  87.33%; FB1:  88.02
         negative: precision:  91.60%; recall:  93.91%; FB1:  92.74  1464
         positive: precision:  63.47%; recall:  46.29%; FB1:  53.54  167